In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

fn = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        fn.append(os.path.join(dirname, filename))
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashion-products/fashion_products.csv


In [2]:
df = pd.read_csv(fn[0])
df

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
...,...,...,...,...,...,...,...,...,...
995,20,996,Shoes,Zara,Women's Fashion,55,1.620081,Black,M
996,42,997,Sweater,Nike,Kids' Fashion,13,1.544464,Green,L
997,9,998,Sweater,Zara,Men's Fashion,47,3.961913,White,L
998,8,999,T-shirt,Zara,Women's Fashion,68,3.089722,Blue,S


In [3]:
df['User ID'].unique().__len__()

100

In [4]:
df.columns

Index(['User ID', 'Product ID', 'Product Name', 'Brand', 'Category', 'Price',
       'Rating', 'Color', 'Size'],
      dtype='object')

In [5]:
import pandas as pd
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Prepare the data for CF (Collaborative Filtering)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['User ID', 'Product ID', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

# Train a Collaborative Filtering model (SVD)
cf_model = SVD()
cf_model.fit(trainset)
cf_predictions = cf_model.test(testset)


In [8]:
# Content-Based Filtering using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
# Combine 'Product Name', 'Brand', and 'Category' to form product descriptions
df['Product_Description'] = df['Product Name'] + ' ' + df['Brand'] + ' ' + df['Category']
product_profiles = tfidf.fit_transform(df['Product_Description'])

# Calculate the cosine similarity between products
similarity_matrix = cosine_similarity(product_profiles)

# Function to get similar products based on content similarity
def get_similar_products(product_id, top_n=5):
    product_index = df[df['Product ID'] == product_id].index[0]
    similar_indices = similarity_matrix[product_index].argsort()[-top_n:][::-1]
    return df.iloc[similar_indices]

In [9]:
def hybrid_recommendation(user_id, top_n=5, alpha=0.5):
    # Get top products from Collaborative Filtering
    cf_scores = []
    for product_id in df['Product ID'].unique():
        prediction = cf_model.predict(user_id, product_id)
        cf_scores.append((product_id, prediction.est))
    
    cf_scores.sort(key=lambda x: x[1], reverse=True)
    cf_top_products = [product for product, score in cf_scores[:top_n]]
    
    # Get top products from Content-Based Filtering based on similar products
    cb_top_products = []
    for product_id in cf_top_products:
        similar_products = get_similar_products(product_id, top_n=top_n)
        cb_top_products.extend(similar_products['Product ID'].values)
    
    # Combine the two sets of recommendations (using alpha for weighted combination)
    hybrid_recommendations = list(set(cf_top_products + cb_top_products))
    
    return hybrid_recommendations[:top_n]

# Example of hybrid recommendations for a user
user_id = 123  # Example User ID
recommended_products = hybrid_recommendation(user_id)
print("Recommended Products:", recommended_products)

Recommended Products: [642, 770, 263, 780, 271]


In [17]:
def hybrid_recommendation(user_id, top_n=5, alpha=0.5):
    # Get top products from Collaborative Filtering
    cf_scores = []
    for product_id in df['Product ID'].unique():
        prediction = cf_model.predict(user_id, product_id)
        cf_scores.append((product_id, prediction.est))
    
    cf_scores.sort(key=lambda x: x[1], reverse=True)
    cf_top_products = [product for product, score in cf_scores[:top_n]]
    
    # Get top products from Content-Based Filtering based on similar products
    cb_top_products = []
    for product_id in cf_top_products:
        similar_products = get_similar_products(product_id, top_n=top_n)
        cb_top_products.extend(similar_products['Product ID'].values)
    
    # Combine the two sets of recommendations (using alpha for weighted combination)
    hybrid_recommendations = list(set(cf_top_products + cb_top_products))
    
    # Get the details of the recommended products
    recommended_product_details = df[df['Product ID'].isin(hybrid_recommendations)]
    
    return recommended_product_details[['Product ID', 'Product Name', 'Brand', 'Category', 'Price', 'Rating', 'Color', 'Size']].head(top_n)

# Example of hybrid recommendations for a user with product details
user_id = 123  # Example User ID
recommended_products = hybrid_recommendation(user_id)
print("Recommended Product Details:")
print(recommended_products)


Recommended Product Details:
     Product ID Product Name Brand         Category  Price    Rating   Color  \
23           24        Jeans  Zara  Women's Fashion     15  2.499756  Yellow   
102         103        Jeans  Zara  Women's Fashion     84  3.649171   White   
104         105        Jeans  Zara  Women's Fashion     52  1.750931  Yellow   
118         119        Jeans  Nike    Men's Fashion     73  2.657925  Yellow   
181         182        Jeans  Zara    Men's Fashion     68  4.358913  Yellow   

    Size  
23    XL  
102   XL  
104    L  
118    S  
181    L  
